In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm

from scipy.interpolate import RegularGridInterpolator
from tqdm import tqdm

Read **STEFAN HEIGLS** data. Data is currently (until 01.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/4abe80f1c550806021f85af8c57c886e).

In [ ]:
f = np.load('turbulentbox.npy')
data = f.copy()
del f

Read **THOMAS PFEILS** data. Data is currently (until 02.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/ebdbc12c85438bd224a5015206840893).

In [ ]:
with np.load('pluto_data.npz') as f:
    data = f['rho']

# Visualization

## My own renderer

In [ ]:
import volrender

In [ ]:
vmax = data.max()
datacube = LogNorm(vmin=vmax * 1e-4, vmax=vmax, clip=True)(data.ravel()).reshape(data.shape).data

In [ ]:
ren = volrender.Renderer(datacube, plot=False)

x0     = ren.transferfunction.x0.copy()
A      = ren.transferfunction.A.copy()
colors = ren.transferfunction.colors.copy()
sigma  = ren.transferfunction.sigma.copy()

In [ ]:
ren.update(60, 90)

In [ ]:
ren.transferfunction.x0            = [0.1, 0.4, 0.99]
ren.transferfunction.sigma         = [0.05, 0.05, 0.1]
ren.transferfunction.colors        = colors
ren.transferfunction.colors[:, -1] = [0.1, 0.2, 0.01]
ren.transferfunction.A             = [0.2, 0.1, 0.01]

ren.render()

img_norm = Normalize()(ren.image)
img_norm[:, :, -1] /= img_norm[:, :, -1].max()

# make the plot
f,ax = plt.subplots(figsize=(4, 4), dpi=150)
pimg = ax.imshow(img_norm)
ax.axis('off');

Same disk different transfer function

In [ ]:
ren.transferfunction.x0     = np.array([0.2, 0.88])
ren.transferfunction.sigma  = np.array([0.05, 0.02])
ren.transferfunction.colors = np.array([
    [1.  , .5  , 0.  , 0.1],
    [.25  , .25  , .75  , 0.1]])
ren.transferfunction.A      = np.array([1, 2])
ren.render()

# make the plot
f,ax = plt.subplots(figsize=(4, 4), dpi=150)
pimg = ax.imshow(Normalize()(ren.image))
ax.axis('off');
ren.image.max()

### Make a movie

In [ ]:
# prepare output folder
frames_path = Path('frames')
if not frames_path.is_dir():
    frames_path.mkdir()
    
# make the plot
f,ax = plt.subplots(figsize=(4, 4), dpi=150)
pimg = ax.imshow(255 * ren.image)
ax.axis('off')
    
# make the loop    
n_angles = 55
phi = np.linspace(0, 90, n_angles)
theta = np.linspace(55, 55, n_angles)
    
for i, (_phi, _theta) in enumerate(tqdm(zip(phi, theta), total=n_angles)):
    image = ren.update(_theta, _phi)
    pimg.set_array(Normalize()(ren.image).data)
    f.savefig(frames_path / f'frame_{i:03d}.jpg', bbox_inches='tight', dpi=200)

In [ ]:
!ffmpeg -y -i {frames_path}/frame_%03d.jpg -c:v libx264 -crf 15 -maxrate 400k -pix_fmt yuv420p -r 20 -bufsize 1835k movie_01.mp4

In [ ]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls>
  <source src="movie_01.mp4" type="video/mp4">
</video>
""")

### Data statistics

Show some statistics of the data

In [ ]:
from scipy.integrate import cumtrapz

f, axs = plt.subplots(1, 2, figsize=(10, 4))

N = datacube.shape[-1]
z = np.linspace(-N / 2, N / 2, N)

ax = axs[0]
ax.plot(z, cumtrapz(datacube, x=z, axis=-1, initial=0).mean(0).mean(0), 'r', lw=3)
for i in range(50):
    ix = np.random.choice(np.arange(data.shape[0]))
    iy = np.random.choice(np.arange(data.shape[1]))
    ax.plot(z, cumtrapz(datacube[ix, iy, :], x=z, initial=0), 'k', alpha=0.1)
ax.set_xlabel('z')

ax = axs[1]
counts, edges, bars = ax.hist(datacube.ravel(), 50)
ax.set_yscale('log')

In [ ]:
!open frames

## Pyvista

In [ ]:
import pyvista

In [ ]:
grid = pyvista.UniformGrid()
grid.dimensions = np.array(data.shape) + 1
grid.spacing = (1, 1, 1)
grid.cell_data["density"] = np.log10(data.flatten(order="F"))

vmax = grid['density'].max()
grid['opac'] = Normalize(vmin=vmax - 1.5, vmax=vmax)(grid['density'])

In [ ]:
pyvista.close_all()

In [ ]:
p = pyvista.Plotter()
p.set_background('white')
p.add_volume(grid, scalars='opac', cmap="viridis", clim=[0, 1], opacity='linear', shade=False, opacity_unit_distance=.010)
p.show()

## YT

In [ ]:
import yt

In [ ]:
plt.style.use({'figure.dpi':150})

In [ ]:
with np.load('pluto_data.npz') as f:
    data = f['rho']

In [ ]:
data_dict = dict(density = (data, "cm**-3"))
bbox = np.array([[-1, 1], [-1, 1], [-1, 1]])
ds = yt.load_uniform_grid(data_dict, data.shape, bbox=bbox)

In [ ]:
slc = yt.SlicePlot(ds, "z", ("gas", "density"))
slc.set_cmap(("gas", "density"), "Blues")
slc.annotate_grids(cmap=None)
slc.show()

In [ ]:
sc = yt.create_scene(ds)

# Get a reference to the VolumeSource associated with this scene
# It is the first source associated with the scene, so we can refer to it
# using index 0.
source = sc[0]

# Set the bounds of the transfer function
source.tfh.set_bounds((1e-23, 1e-20))

# set that the transfer function should be evaluated in log space
source.tfh.set_log(True)

# Make underdense regions appear opaque
source.tfh.grey_opacity = False

# Plot the transfer function, along with the CDF of the density field to
# see how the transfer function corresponds to structure in the CDF
source.tfh.plot("transfer_function.png", profile_field=("gas", "density"))

# save the image, flooring especially bright pixels for better contrast
sc.save("rendering.pdf", sigma_clip=6.0)

In [ ]:
!open rendering.pdf